In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-midterm-p5/Feature_engineering.py
/kaggle/input/2022-ml-midterm-p5/activity_labels.txt
/kaggle/input/2022-ml-midterm-p5/features_info.txt
/kaggle/input/2022-ml-midterm-p5/data_loader.py
/kaggle/input/2022-ml-midterm-p5/features.txt
/kaggle/input/2022-ml-midterm-p5/submit.csv
/kaggle/input/2022-ml-midterm-p5/RawData/acc_exp28_user14.txt
/kaggle/input/2022-ml-midterm-p5/RawData/acc_exp45_user22.txt
/kaggle/input/2022-ml-midterm-p5/RawData/acc_exp15_user08.txt
/kaggle/input/2022-ml-midterm-p5/RawData/acc_exp05_user03.txt
/kaggle/input/2022-ml-midterm-p5/RawData/gyro_exp13_user07.txt
/kaggle/input/2022-ml-midterm-p5/RawData/acc_exp43_user21.txt
/kaggle/input/2022-ml-midterm-p5/RawData/acc_exp11_user06.txt
/kaggle/input/2022-ml-midterm-p5/RawData/gyro_exp26_user13.txt
/kaggle/input/2022-ml-midterm-p5/RawData/acc_exp39_user19.txt
/kaggle/input/2022-ml-midterm-p5/RawData/gyro_exp05_user03.txt
/kaggle/input/2022-ml-midterm-p5/RawData/label_train.txt
/kaggle/input/2022-ml-

In [2]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
import scipy as sp
# Importing Pandas Library 
from glob import glob
# import display() for better visualitions of DataFrames and arrays
from IPython.display import display
# import pyplot for plotting
import matplotlib.pyplot as plt
plt.style.use('bmh') # for better plots
# import data_loader for data loading
from data_loader import import_raw_signals, import_labels_file,normalize5,normalize2

In [4]:
Raw_data_paths = sorted(glob("/kaggle/input/2022-ml-midterm-p5/RawData/*"))
Raw_acc_paths=Raw_data_paths[0:61]
Raw_gyro_paths=Raw_data_paths[61:122]

In [5]:
raw_dic={}
raw_acc_columns=['acc_X','acc_Y','acc_Z']
raw_gyro_columns=['gyro_X','gyro_Y','gyro_Z']
for path_index in range(0,61):
        key= Raw_data_paths[path_index][-16:-4]
        raw_acc_data_frame=import_raw_signals(Raw_data_paths[path_index],raw_acc_columns)
        raw_gyro_data_frame=import_raw_signals(Raw_data_paths[path_index+61],raw_gyro_columns)
        raw_signals_data_frame=pd.concat([raw_acc_data_frame, raw_gyro_data_frame], axis=1)
        raw_dic[key]=raw_signals_data_frame

In [6]:
train_raw_labels_columns=['experiment_number_ID','user_number_ID','activity_number_ID','Label_start_point','Label_end_point']
test_raw_labels_columns=['experiment_number_ID','user_number_ID','Label_start_point','Label_end_point']

test_labels_path=Raw_data_paths[122]
train_labels_path=Raw_data_paths[123]

train_Labels_Data_Frame=import_labels_file(train_labels_path,train_raw_labels_columns)
test_Labels_Data_Frame=import_labels_file(test_labels_path,test_raw_labels_columns)

In [7]:
from scipy.signal import medfilt

def median(signal):
    array=np.array(signal)   
    med_filtered=sp.signal.medfilt(array, kernel_size=3)
    return  med_filtered

In [8]:
from scipy.fftpack import fft  
from scipy.fftpack import fftfreq
from scipy.fftpack import ifft
import math 

sampling_freq = 50
nyq=sampling_freq/float(2)
freq1 = 0.3
freq2 = 20

# Function name: components_selection_one_signal

# Inputs: t_signal:1D numpy array (time domain signal); 

# Outputs: (total_component,t_DC_component , t_body_component, t_noise) 
#           type(1D array,1D array, 1D array)

# cases to discuss: if the t_signal is an acceleration signal then the t_DC_component is the gravity component [Grav_acc]
#                   if the t_signal is a gyro signal then the t_DC_component is not useful
# t_noise component is not useful
# if the t_signal is an acceleration signal then the t_body_component is the body's acceleration component [Body_acc]
# if the t_signal is a gyro signal then the t_body_component is the body's angular velocity component [Body_gyro]

def components_selection_one_signal(t_signal,freq1,freq2):
    t_signal=np.array(t_signal)
    t_signal_length=len(t_signal)
    f_signal=fft(t_signal)
    freqs=np.array(sp.fftpack.fftfreq(t_signal_length, d=1/float(sampling_freq)))# frequency values between [-25hz:+25hz]
    
    # DC_component: f_signal values having freq between [-0.3 hz to 0 hz] and from [0 hz to 0.3hz] 
    #                                                             (-0.3 and 0.3 are included)
    
    # noise components: f_signal values having freq between [-25 hz to 20 hz[ and from ] 20 hz to 25 hz] 
    #                                                               (-25 and 25 hz inculded 20hz and -20hz not included)
    
    # selecting body_component: f_signal values having freq between [-20 hz to -0.3 hz] and from [0.3 hz to 20 hz] 
    #                                                               (-0.3 and 0.3 not included , -20hz and 20 hz included)
    
    
    f_DC_signal=[] # DC_component in freq domain
    f_body_signal=[] # body component in freq domain numpy.append(a, a[0])
    f_noise_signal=[] # noise in freq domain
    
    for i in range(len(freqs)):# iterate over all available frequencies
        
        # selecting the frequency value
        freq=freqs[i]
        
        # selecting the f_signal value associated to freq
        value= f_signal[i]
        
        # Selecting DC_component values 
        if abs(freq)>0.3:# testing if freq is outside DC_component frequency ranges
            f_DC_signal.append(float(0)) # add 0 to  the  list if it was the case (the value should not be added)                                       
        else: # if freq is inside DC_component frequency ranges 
            f_DC_signal.append(value) # add f_signal value to f_DC_signal list
    
        # Selecting noise component values 
        if (abs(freq)<=20):# testing if freq is outside noise frequency ranges 
            f_noise_signal.append(float(0)) # # add 0 to  f_noise_signal list if it was the case 
        else:# if freq is inside noise frequency ranges 
            f_noise_signal.append(value) # add f_signal value to f_noise_signal

        # Selecting body_component values 
        if (abs(freq)<=0.3 or abs(freq)>20):# testing if freq is outside Body_component frequency ranges
            f_body_signal.append(float(0))# add 0 to  f_body_signal list
        else:# if freq is inside Body_component frequency ranges
            f_body_signal.append(value) # add f_signal value to f_body_signal list
    
    ################### Inverse the transformation of signals in freq domain ########################
    # applying the inverse fft(ifft) to signals in freq domain and put them in float format
    t_DC_component= ifft(np.array(f_DC_signal)).real
    t_body_component= ifft(np.array(f_body_signal)).real
    t_noise=ifft(np.array(f_noise_signal)).real
    
    total_component=t_signal-t_noise # extracting the total component(filtered from noise) 
                                     #  by substracting noise from t_signal (the original signal).
    
    # return outputs mentioned earlier
    return (total_component,t_DC_component,t_body_component,t_noise)

In [9]:
import math
def mag_3_signals(x,y,z): # Euclidian magnitude
    return [math.sqrt((x[i]**2+y[i]**2+z[i]**2)) for i in range(len(x))]

In [10]:
dt=0.02 # dt=1/50=0.02s time duration between two rows
# Input: 1D array with lenght=N (N:unknown)
# Output: 1D array with lenght=N-1
def jerk_one_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [11]:
time_sig_dic={}
raw_dic_keys=sorted(raw_dic.keys())

import tqdm
for key in tqdm.tqdm(raw_dic_keys):
    raw_df=raw_dic[key]
    time_sig_df=pd.DataFrame()
    
    for column in raw_df.columns:
        t_signal=np.array(raw_df[column])
        med_filtred=median(t_signal)
        
        if 'acc' in column:
            _,grav_acc,body_acc,_=components_selection_one_signal(med_filtred,freq1,freq2)
            body_acc_jerk=jerk_one_signal(body_acc)
            time_sig_df['t_body_'+column]=body_acc[:-1]
            time_sig_df['t_grav_'+column]= grav_acc[:-1]
            time_sig_df['t_body_acc_jerk_'+column[-1]]=body_acc_jerk
        elif 'gyro' in column:
            _,_,body_gyro,_=components_selection_one_signal(med_filtred,freq1,freq2)
            body_gyro_jerk=jerk_one_signal(body_gyro)
            time_sig_df['t_body_gyro_'+column[-1]]=body_gyro[:-1]
            time_sig_df['t_body_gyro_jerk_'+column[-1]]=body_gyro_jerk
            
    new_columns_ordered=['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                      't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                      't_body_acc_jerk_X','t_body_acc_jerk_Y','t_body_acc_jerk_Z',
                      't_body_gyro_X','t_body_gyro_Y','t_body_gyro_Z',
                      't_body_gyro_jerk_X','t_body_gyro_jerk_Y','t_body_gyro_jerk_Z']
        
    ordered_time_sig_df=pd.DataFrame()
        
    for col in new_columns_ordered:
        ordered_time_sig_df[col]=time_sig_df[col]
        
    for i in range(0,15,3):
        mag_col_name=new_columns_ordered[i][:-1]+'mag'
        col0=np.array(ordered_time_sig_df[new_columns_ordered[i]]) # copy X_component
        col1=ordered_time_sig_df[new_columns_ordered[i+1]] # copy Y_component
        col2=ordered_time_sig_df[new_columns_ordered[i+2]] # copy Z_component
        mag_signal=mag_3_signals(col0,col1,col2)
        ordered_time_sig_df[mag_col_name]=mag_signal
        
    time_sig_dic[key]=ordered_time_sig_df

100%|██████████| 61/61 [01:28<00:00,  1.46s/it]


In [12]:
def Windowing_type(time_sig_dic,Labels_Data_Frame):
    columns=time_sig_dic['exp01_user01'].columns
    window_ID=0
    time_dictionary_window={}
    BA_array=np.array(Labels_Data_Frame)
    
    for line in tqdm.tqdm(BA_array):
        file_key= 'exp' + normalize2(int(line[0]))  +  '_user' + normalize2(int(line[1]))
        
        if line.shape[0] == 5 :
          act_ID=line[2]
          start_point=line[3]
          end_point = line[4]
        else :
          act_ID='None'
          start_point = line[2]
          end_point = line[3]
        
        for cursor in range(start_point,end_point-127,64):
            end_point=cursor+128
            data=np.array(time_sig_dic[file_key].iloc[cursor:end_point])
            window=pd.DataFrame(data=data,columns=columns)
            key='t_W'+normalize5(window_ID)+'_'+file_key+'_act'+normalize2(act_ID)
            time_dictionary_window[key]=window
            window_ID=window_ID+1
    
    return time_dictionary_window 

In [13]:
train_time_dictionary_window  = Windowing_type(time_sig_dic,train_Labels_Data_Frame)
test_time_dictionary_window  = Windowing_type(time_sig_dic,test_Labels_Data_Frame)

100%|██████████| 258/258 [00:01<00:00, 185.21it/s]


In [14]:
train_window = train_time_dictionary_window[sorted(train_time_dictionary_window.keys())[0]]

In [15]:
from scipy import fftpack
from numpy.fft import *

def fast_fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    
    return amplitude_f_signal

def fast_fourier_transform(t_window):
    f_window=pd.DataFrame()
    for column in t_window.columns:
        if 'grav' not in column:
            t_signal=np.array(t_window[column])
            f_signal= np.apply_along_axis(fast_fourier_transform_one_signal,0,t_signal)
            f_window["f_"+column[2:]]=f_signal
    return f_window

In [16]:
train_frequent_dictionary_window = {'f'+key[1:] : train_t_df.pipe(fast_fourier_transform) for key, train_t_df in tqdm.tqdm(train_time_dictionary_window.items())}
test_frequent_dictionary_window = {'f'+key[1:] : test_t_df.pipe(fast_fourier_transform) for key, test_t_df in tqdm.tqdm(test_time_dictionary_window.items())}

100%|██████████| 3116/3116 [00:25<00:00, 120.33it/s]


In [17]:
train_window = train_frequent_dictionary_window[sorted(train_frequent_dictionary_window.keys())[0]]

In [18]:
# -------------------------------------
# [Empty Module #1] Feature Engineering
# -------------------------------------

# -------------------------------------
# Feature Engineering
# -------------------------------------
# 목적: 제공된 36개의 시퀀스 도메인 데이터를 기반으로 유의미한 피처를 추출한다.
# 입력인자: 시간(time) 도메인 Feature 20개 , 주파수(frequency) 도메인 Feature 16개
# 출력인자: 분류모델 학습을 위한 Feature
# -------------------------------------

# ------------------------------------------------------------
# 구현 가이드라인 - 논문에서 제안하는 Feature Engineering 방법
# ------------------------------------------------------------
#
# mean(): Mean value
# std(): Standard deviation
# mad(): Median absolute deviation 
# max(): Largest value in array
# min(): Smallest value in array
# sma(): Signal magnitude area
# energy(): Energy measure. Sum of the squares divided by the number of values. 
# iqr(): Interquartile range 
# entropy(): Signal entropy
# arCoeff(): Autorregresion coefficients with Burg order equal to 4
# correlation(): correlation coefficient between two signals
# maxInds(): index of the frequency component with largest magnitude
# meanFreq(): Weighted average of the frequency components to obtain a mean frequency
# skewness(): skewness of the frequency domain signal 
# kurtosis(): kurtosis of the frequency domain signal 
# bandsEnergy(): Energy of a frequency interval within the 64 bins of the FFT of each window.
# angle(): Angle between to vectors.

import sys


# Time domain Feature Extract function

from Feature_engineering import mean_axial,std_axial,mad_axial,max_axial,min_axial, t_sma_axial, t_energy_axial,IQR_axial,entropy_axial, t_arburg_axial, t_corr_axial
from Feature_engineering import mean_mag,std_mag,mad_mag,max_mag,min_mag,t_sma_mag,t_energy_mag,IQR_mag,entropy_mag,t_arburg_mag

# Frequency domain Feature Extract function
from Feature_engineering import f_sma_axial,f_energy_axial,f_max_Inds_axial,f_mean_Freq_axial,f_skewness_and_kurtosis_axial,f_all_bands_energy_axial
from Feature_engineering import f_sma_mag,f_energy_mag,f_max_Inds_mag,f_mean_Freq_mag,f_skewness_mag,f_kurtosis_mag

# Additional Feature Extract function
from Feature_engineering import angle_features

def feature_extractor(time_dictionary,freq_dictionary, condition='train') :
    
    
    if condition is 'train' :
        total_data = []
        total_label = []
    elif condition is 'test' :
        total_data = []
        
    for i in tqdm.tqdm(range(len(time_dictionary))) :
        
        time_key = sorted(time_dictionary.keys())[i]
        freq_key = sorted(freq_dictionary.keys())[i]
        
        time_window = time_dictionary[time_key]
        freq_window = freq_dictionary[freq_key]
        
        if condition is 'train' :
          window_user_id= int(time_key[-8:-6]) # extract the user id from window's key
          window_activity_id=int(time_key[-2:]) # extract the activity id from the windows key
        elif condition is 'test' :
          window_user_id= int(time_key[-10:-8]) # extract the user id from window's key
          window_activity_id= 0
        else :
            print("Error")
            sys.exit()
            break;
            
        ##################################################################################
        
        
        # Time domain - Feature extractor - Part 1. axial(X,Y,Z) Features 
        
        #[0,1,2] : 't_body_acc_X', 't_body_acc_Y', 't_body_acc_Z'
        #[3,4,5] : 't_grav_acc_X','t_grav_acc_Y', 't_grav_acc_Z'
        #[6,7,8] : 't_body_acc_jerk_X','t_body_acc_jerk_Y', 't_body_acc_jerk_Z'
        #[9,10,11] : 't_body_gyro_X','t_body_gyro_Y', 't_body_gyro_Z'
        #[12,13,14] : 't_body_gyro_jerk_X', 't_body_gyro_jerk_Y', 't_body_gyro_jerk_Z'
        
        axial_columns = time_window.columns[0:15]
        axial_df = time_window[axial_columns] # X,Y,Z
        
        time_axial_features = []
        
        for col in range(0,15,3) : 
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 time_3axial_vector 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            curr_col = axial_columns[col:col+3]
            curr_df = axial_df[curr_col]
            
            mean_vector = mean_axial(curr_df)
            std_vector = std_axial(curr_df)
            mad_vector = mad_axial(curr_df)
            max_vector = max_axial(curr_df)
            min_vector = min_axial(curr_df)
            sma_value = t_sma_axial(curr_df)
            energy_vector = t_energy_axial(curr_df)
            IQR_vector = IQR_axial(curr_df)
            entropy_vector = entropy_axial(curr_df)
            AR_vector = t_arburg_axial(curr_df)
            corr_vector = t_corr_axial(curr_df)
            # 40 value per each 3-axial signals
            time_3axial_vector = mean_vector + std_vector + mad_vector + \
                                 max_vector + min_vector + [sma_value] + \
                                 energy_vector + IQR_vector + entropy_vector + \
                                 AR_vector + corr_vector
            
            # append these features to the global list of features
            time_axial_features= time_axial_features+time_3axial_vector
        
        ##################################################################################
        
        # Time domain - Feature extractor - Part 2. Magnitude Features 
        
        #[15]'t_body_acc_mag'
        #[16]'t_grav_acc_mag'
        #[17]'t_body_acc_jerk_mag'
        #[18]'t_body_gyro_mag'
        #[19]'t_body_gyro_jerk_mag'
        
        mag_columns = time_window.columns[15:]
        mag_columns = time_window[mag_columns]
        
        time_mag_features = []
        
        for ci, col in enumerate(mag_columns) :
            
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 col_mag_values 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
   
            mean_value = mean_mag(mag_columns[col])
            std_value = std_mag(mag_columns[col])
            mad_value = mad_mag(mag_columns[col])
            max_value = max_mag(mag_columns[col])
            min_value = min_mag(mag_columns[col])
            sma_value = f_sma_mag(mag_columns[col])
            energy_value = t_energy_mag(mag_columns[col])
            IQR_value = IQR_mag(mag_columns[col])
            entropy_value = entropy_mag(mag_columns[col])
            
            curr_col = axial_columns[int(ci*3):int(ci*3)+3]
            curr_df = axial_df[curr_col]
            AR_vector = t_arburg_axial(curr_df)
            
            # 13 value per each t_mag_column
            col_mag_values = [mean_value, std_value, mad_value, max_value, min_value, sma_value, 
                              energy_value,IQR_value, entropy_value]+ AR_vector

            # col_mag_values will be added to the global list
            time_mag_features= time_mag_features+ col_mag_values

        
        ##################################################################################
        
        # Frequency domain - Feature extractor - Part 1. axial(X,Y,Z) Features 
        
        #[0,1,2] : 'f_body_acc_X', 'f_body_acc_Y', 'f_body_acc_Z'
        #[3,4,5] : 'f_body_acc_jerk_X','f_body_acc_jerk_Y', 'f_body_acc_jerk_Z'
        #[6,7,8] : 'f_body_gyro_X','f_body_gyro_Y', 'f_body_gyro_Z'
        #[9,10,11] : 'f_body_gyro_jerk_X','f_body_gyro_jerk_Y', 'f_body_gyro_jerk_Z'
        
        axial_columns=freq_window.columns[0:12]
        axial_df=freq_window[axial_columns]
        freq_axial_features=[]
        
        for col in range(0,12,3) :         
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 freq_3axial_features 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            
            curr_col = axial_columns[col:col+3]
            curr_df = axial_df[curr_col]
            
            mean_vector = mean_axial(curr_df)
            std_vector = std_axial(curr_df)
            mad_vector = mad_axial(curr_df)
            max_vector = max_axial(curr_df)
            min_vector = min_axial(curr_df)
            sma_value = f_sma_axial(curr_df)
            energy_vector = f_energy_axial(curr_df)
            IQR_vector = IQR_axial(curr_df)
            entropy_vector = entropy_axial(curr_df)
            max_inds_vector = f_max_Inds_axial(curr_df)
            mean_Freq_vector = f_mean_Freq_axial(curr_df)
            skewness_and_kurtosis_vector = f_skewness_and_kurtosis_axial(curr_df)
            bands_energy_vector = f_all_bands_energy_axial(curr_df)
            
            freq_3axial_features = mean_vector +std_vector + mad_vector + max_vector + min_vector + [sma_value] + energy_vector + IQR_vector + entropy_vector + max_inds_vector + mean_Freq_vector + skewness_and_kurtosis_vector + bands_energy_vector
            freq_axial_features = freq_axial_features+ freq_3axial_features
        
        ##################################################################################
        
        # Frequency domain - Feature extractor - Part 2. Magnitude Features
        
        #[12]'f_body_acc_mag'
        #[13]'f_body_acc_jerk_mag'
        #[14]'f_body_gyro_mag'
        #[15]'f_body_gyro_jerk_mag'
        
        mag_columns=freq_window.columns[12:]
        mag_columns=freq_window[mag_columns]
        
        freq_mag_features = []
        
        for col in mag_columns:
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 col_mag_values에 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            mean_value = mean_mag(mag_columns[col])
            std_value = std_mag(mag_columns[col])
            mad_value = mad_mag(mag_columns[col])
            max_value = max_mag(mag_columns[col])
            min_value = min_mag(mag_columns[col])
            sma_value = t_sma_mag(mag_columns[col])
            energy_value = t_energy_mag(mag_columns[col])
            IQR_value = IQR_mag(mag_columns[col])
            entropy_value = entropy_mag(mag_columns[col])
            max_Inds_value = f_max_Inds_mag(mag_columns[col])
            mean_Freq_value = f_mean_Freq_mag(mag_columns[col])
            skewness_value = f_skewness_mag(mag_columns[col])
            kurtosis_value = f_kurtosis_mag(mag_columns[col])
            # 13 value per each t_mag_column
            col_mag_values = [mean_value, std_value, mad_value, max_value, 
                              min_value, sma_value, energy_value,IQR_value, 
                              entropy_value, max_Inds_value, mean_Freq_value,
                              skewness_value, kurtosis_value ]
            
            freq_mag_features= freq_mag_features+ col_mag_values
        
        ##################################################################################
        
        # Time domain - Feature extractor - Part 3. Additional Features 
        
        additional_features = angle_features(time_window)
                
        ##################################################################################
        
        total_features = time_axial_features + time_mag_features + freq_axial_features + freq_mag_features + additional_features
        
        total_data.append(total_features)
        if condition is 'train' :
            total_label.append(window_activity_id)
    
    total_data = np.array(total_data)
    if condition is 'train' :
        total_label = np.array(total_label)
    
    if condition is 'train' :
        return total_data, total_label
    elif condition is 'test' :
        return total_data

In [19]:
train_data, train_label = feature_extractor(train_time_dictionary_window,train_frequent_dictionary_window,condition='train')
test_data = feature_extractor(test_time_dictionary_window,test_frequent_dictionary_window,condition='test')

100%|██████████| 3116/3116 [02:30<00:00, 20.76it/s]


In [20]:
# -------------------------------------
# [Empty Module #2] Data Normalization
# -------------------------------------

# -------------------------------------
# Data Normalization
# -------------------------------------
# 목적: 앞서 구축한 train,test 셋에 대한 Feature를 정규화한다.
# 입력인자: train 셋에서 추출된 Feature, test 셋에서 추출된 Feature
# 출력인자: 정규화된 Feature Vector
# -------------------------------------


from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_data)
train_norm_data = scaler.transform(train_data)
test_norm_data = scaler.transform(test_data)
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# sklearn에서 제공하는 MinMaxScaler를 사용해 데이터 정규화를 진행한다.
# (MinMaxScaler가 아닌 다른 정규화를 사용할 수 있다.)
# ------------------------------------------------------------

In [21]:
# -------------------------------------
# [Empty Module #3] RandomForest를 이용한 분류
# -------------------------------------

# -------------------------------------
# SVC를 이용한 분류
# -------------------------------------
# 목적: 앞서 완성한 train/test Feature를 RandomForest를 이용해 분류한다.
# 입력인자: Feature vector(train/test)
# 출력인자: 분류결과
# -------------------------------------

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=200)
clf.fit(train_norm_data, train_label)
y_pred = clf.predict(test_norm_data)

# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# sklearn에서 제공하는 RandomForest를 사용해 데이터 정규화를 진행한다.
# (RandomForest를가 아닌 다른 분류모델을 사용할 수 있다.)
# ------------------------------------------------------------

In [22]:
submit_csv = pd.read_csv('/kaggle/input/2022-ml-midterm-p5/submit.csv')
submit_csv['Label'] = y_pred
submit_csv.to_csv("submit.csv", index=False)